In [48]:
%use dataframe
%use kandy

In [49]:
enum class StartType {
    COLD,
    WARM
}

data class InvokeResult(
    val functionName: String,
    val requestId: String,
    val duration: Int,
    val billedDuration: Int,
    val initDuration: Int? = null,
    val restoreDuration: Int? = null,
    val billedRestoreDuration: Int? = null,
    val memorySize: Int,
    val maxMemoryUsed: Int,
    val timestamp: Long,
    val startType: StartType
)

# Wczytywanie i preprocessing danych

In [50]:
val rawResultsDataframe = DataFrame.read("results.json")

In [51]:
import org.jetbrains.kotlinx.dataframe.DataFrame

val resultsDataframe = rawResultsDataframe
    .add("startType") {
        if (it["initDuration"] == null && it["restoreDuration"] == null) {
            StartType.WARM
        } else {
            StartType.COLD
        }
    }
    .update(InvokeResult::functionName) {
        it.replace("mte-tested-", "")
            .split("-")
            .dropLast(1)
            .filter {
                it != "function" && it.isNotBlank()
            }
            .map { it.trim() }
            .map {
                if (it == "jar") {
                    "JVM"
                } else if (it == "js") {
                    "JS"
                } else {
                    it.replaceFirstChar { it.uppercase() }
                }
            }
            .joinToString(" ")
    }
    .sortBy { it.functionName }
    .cast<InvokeResult>()

resultsDataframe

requestId,duration,billedDuration,memorySize,maxMemoryUsed,restoreDuration,billedRestoreDuration,timestamp,functionName,initDuration,startType
7d602f8a-167f-4c6c-a77d-f75936ec3f17,589.500000,590,256,143,null,null,1747012931179,Java JVM,1434.860000,COLD
1372ed0a-3fa9-4e38-b969-125443b40b1d,2.600000,3,256,143,null,null,1747012936214,Java JVM,null,WARM
baad196f-4f23-484c-8571-cf0e99d8837c,22.660000,23,256,143,null,null,1747012941259,Java JVM,null,WARM
e8a0a2ca-e211-456d-8535-b5665d0bed4c,2.250000,3,256,143,null,null,1747012946288,Java JVM,null,WARM
00cd42b3-d5ba-4d9e-b1f1-b12e7e6a17b6,2.240000,3,256,143,null,null,1747012951315,Java JVM,null,WARM
a90f6fa6-0507-423d-a1ab-99a41bc6556a,17.920000,18,256,143,null,null,1747012956359,Java JVM,null,WARM
a4326847-b92a-48ad-a893-04f1b23f3ccd,2.600000,3,256,143,null,null,1747012961385,Java JVM,null,WARM
3c8b514a-f897-43e8-a5e0-710d2086a8ca,2.350000,3,256,143,null,null,1747012966408,Java JVM,null,WARM
ffedb27b-df85-40d9-9063-9a583c2764db,5.200000,6,256,143,null,null,1747012971438,Java JVM,null,WARM
bab54a45-bfcc-41e7-aa5b-d330aa907ceb,2.120000,3,256,143,null,null,1747012976461,Java JVM,null,WARM


# Metody do wykresów

In [52]:
import org.jetbrains.kotlinx.statistics.kandy.layers.context.BoxplotLayerBuilder

val defaultBoxplotStyling: BoxplotLayerBuilder<Any?>.() -> Unit = {
    boxes {
        fatten = 0.5
        alpha = 0.6
        borderLine.color(Stat.x)
    }

    outliers {
        color(Stat.x)
    }
}

In [53]:
val coldStartsDataFrame = resultsDataframe
    .filter { it.startType == StartType.COLD }
    .filter { !it.functionName.contains("snapstart") || it.initDuration == null }
    .add("totalDuration") {
        it.duration.toDouble() + (it.initDuration?.toDouble() ?: 0.0) + (it.restoreDuration?.toDouble() ?: 0.0)
    }

val warmStartsDataframe = resultsDataframe
    .filter { it.startType == StartType.WARM }

In [54]:
fun plotColdStartsBoxplot(memorySize: Int) =
    coldStartsDataFrame
        .filter { row ->
            row.memorySize == memorySize
        }
        .also { df ->
            val memorySizeExists = df.rows().toList().size > 0
            if (!memorySizeExists) {
                throw IllegalArgumentException("Memory size $memorySize doesn't exist")
            }
        }
        .plot {
            boxplot(x = InvokeResult::functionName.name, y = "totalDuration") {
                layout.xAxisLabel = "Funkcja"
                layout.yAxisLabel = "Czas wykonania [ms]"
                layout.size = 1200 to 600

                defaultBoxplotStyling()
            }
        }

In [64]:
fun plotWarmStartsBoxplot(memorySize: Int) =
    warmStartsDataframe
        .filter {
            it[InvokeResult::memorySize.name] == memorySize
        }
        .also { df ->
            val memorySizeExists = df.rows().toList().size > 0
            if (!memorySizeExists) {
                throw IllegalArgumentException("Memory size $memorySize doesn't exist")
            }
        }
        .plot {
            boxplot(x = InvokeResult::functionName.name, y = "duration") {
                layout.xAxisLabel = "Funkcja"
                layout.yAxisLabel = "Czas wykonania [ms]"
                layout.size = 1200 to 600

                defaultBoxplotStyling()
            }
        }

## Analiza zimnych startów

In [56]:
plotColdStartsBoxplot(memorySize = 128)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="qz3Zve"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"min":[2588.05,2041.38,549.12,587.02,4613.280000000001,5253.27,274.61],
"middle":[2735.5,2473.3050000000003,600.22,716.08,4943.955,5937.41,323.46],
"max":[2909.62,3034.9799999999996,637.04,836.55,5279.88,6277.700000000001,370.12],
"lower":[2694.5175,2351.1000000000004,586.49,664.84,4841.6,5719.0225,310.36],
"upper":[2792.54,2627.64,611.765,735.4,5024.620000000001,6063.179999999999,336.35],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java Native","Java Native","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JVM","Kotlin JVM","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native"],
"y":[2413.41,2406.87,2428.04,2513.17,2496.7799999999997,2524.09,1910.69,1886.82,1736.75,1760.34,1699.54,1774.2,1795.12,1931.8200000000002,3345.6000000000004,3175.7700000000004,522.3399999999999,528.4300000000001,350.33,320.98,330.03,319.52,3745.98,4529.429999999999,4875.07,4930.759999999999,377.15999999999997,378.89,395.53,387.79999999999995,384.77,385.20000000000005]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("qz3Zve");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 4,500 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 5,500 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 6,500 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 

In [57]:
plotColdStartsBoxplot(memorySize = 256)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="b00DBH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[2196.4,1274.12,422.0,430.08500000000004,2449.1800000000003,3239.765,198.43],
"min":[1966.54,1095.24,371.71,359.2,2134.06,2697.5299999999997,153.28],
"max":[2363.8999999999996,1412.51,440.94,475.28999999999996,2767.2599999999998,3580.42,253.88],
"lower":[2122.8,1221.59,405.95000000000005,406.8425,2367.3199999999997,3081.0024999999996,188.70499999999998],
"upper":[2231.92,1307.6025,429.145,438.6925,2532.8050000000003,3339.2200000000003,216.445],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java Native","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native"],
"y":[1942.1,1931.67,1920.69,1893.62,1938.67,1092.43,1080.8,1039.34,2175.6400000000003,1514.48,1050.57,1064.76,1082.42,2242.1,370.30999999999995,370.0,488.19,2062.87,1959.93,2053.5699999999997,1967.43,2992.9300000000003,1997.1699999999998,2041.8999999999999,2627.63,2486.72,279.08,261.26,262.37,263.99,267.49,285.93,259.76]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("b00DBH");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotl

In [59]:
plotColdStartsBoxplot(memorySize = 512)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Re8QYr"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"min":[1499.15,723.89,285.9,249.99,1162.75,1508.1,94.83],
"middle":[1864.82,887.565,343.655,291.69,1325.1799999999998,1997.8700000000001,135.35],
"max":[2024.9,969.5799999999999,375.25,325.14,1433.84,2362.06,192.02],
"lower":[1741.82,846.435,313.58500000000004,275.595,1272.6125,1816.8874999999998,128.26999999999998],
"upper":[1923.24,930.5799999999999,350.87,300.14,1360.3500000000001,2077.5424999999996,155.32999999999998],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native"],
"y":[1466.56,708.99,704.0,698.78,1841.94,668.63,1110.69,1569.3999999999999,199.52999999999997,210.58,227.39999999999998,1783.03,1407.8999999999999,1413.84,1390.0600000000002,1350.8400000000001,1297.35,223.76,256.18,206.21,214.86,198.56,195.99]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("Re8QYr");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

In [60]:
plotColdStartsBoxplot(memorySize = 1024)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="DRz0nF"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"min":[1462.81,593.07,237.31,185.62,692.01,923.4000000000001,81.53],
"middle":[1681.09,719.9649999999999,307.39,230.375,760.33,1417.63,108.99],
"max":[1827.65,819.5699999999999,318.92,268.62,826.1299999999999,1682.8400000000001,182.99],
"upper":[1729.82,748.56,313.47,239.27,781.96,1528.7050000000002,135.19],
"lower":[1549.17,686.0649999999999,279.835,215.3875,733.315,1256.93,102.41],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java Native","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin Native"],
"y":[588.74,589.2,1731.0,589.07,1703.5900000000001,392.29,504.57,863.51,910.4200000000001,503.83000000000004,512.36,813.39,733.77,817.66,823.31,753.05,778.31,189.1]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("DRz0nF");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Native

In [61]:
plotColdStartsBoxplot(memorySize = 2048)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="btTMKp"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"totalDuration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"min":[1372.51,544.07,243.15,173.82,445.59000000000003,885.87,70.42],
"middle":[1472.77,712.26,297.53999999999996,203.985,493.75,1351.335,99.505],
"max":[1539.26,849.3000000000001,325.21,223.1,534.31,1618.2,145.47],
"upper":[1490.06,751.11,301.57,210.03,504.87,1458.2375,117.0175],
"lower":[1437.69,668.05,260.8575,193.66,481.14,1190.0025,92.615],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native"],
"y":[1131.29,1093.91,1341.0,1586.88,1357.02,1347.3999999999999,429.85,463.34,539.36,525.77,1512.11,878.43,944.52,953.5,998.5999999999999,912.54,976.91,1527.8999999999999,1375.75,442.07,592.91,443.46000000000004,858.73,607.6800000000001,581.87,600.1600000000001,551.74,552.8399999999999,665.26,687.1099999999999,637.9,767.75,758.6800000000001,495.37,497.71000000000004,159.04,154.01,156.8,182.46,167.2,159.95]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("btTMKp");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 


## Analiza ciepłych startów

In [65]:
plotWarmStartsBoxplot(128)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="oWYyDf"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"min":[1.9,2.4,0.99,1.56,1.83,6.65,2.09],
"middle":[14.55,35.86,1.13,2.5599999999999996,10.32,44.69,7.08],
"max":[37.56,75.27,1.57,68.55,30.61,137.37,27.62],
"lower":[8.9575,27.770000000000003,1.08,1.8325,2.01,30.5075,2.285],
"upper":[22.96,50.705,1.28,30.665,14.65,75.6975,14.185],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin Native"],
"y":[55.51,72.03,1.72,1.76,1.68,1.61,1.77,156.95,145.45,112.5,87.47,99.69,91.33,300.0,289.51,132.92,89.46,315.0,136.01,372.0,112.69,290.28,90.54,73.95,130.75,143.73,116.06,84.73,177.95,176.46,149.89,183.88,34.48,114.84,74.4,39.31,37.53]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("oWYyDf");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 350 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Native

In [66]:
plotWarmStartsBoxplot(256)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vHYGcV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[2.16,11.95,1.25,10.24,2.13,13.41,2.06],
"min":[1.8,2.28,0.92,2.0,1.76,2.0,1.82],
"max":[2.97,25.55,77.95,59.54,5.62,43.3,2.37],
"lower":[2.01,2.565,1.09,2.335,1.96,6.625,1.98],
"upper":[2.535,15.445,38.77,33.56999999999999,3.465,22.085,2.215],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native","Kotlin Native","Kotlin Native"],
"y":[22.66,17.92,5.2,20.43,3.38,7.76,10.8,124.4,176.94,128.41,134.17,154.78,87.84,7.39,30.16,13.49,10.37,14.75,19.19,13.32,11.5,17.61,45.76,13.75,4.4,2.63]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("vHYGcV");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 160 
 
 
 
 
 
 
 180 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Native

In [67]:
plotWarmStartsBoxplot(512)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="tqcgHG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[2.37,2.8600000000000003,1.3,1.9649999999999999,2.03,2.935,6.8],
"min":[2.09,2.18,1.07,1.56,1.64,2.0,2.03],
"max":[2.96,4.13,1.56,4.46,3.58,6.8,22.64],
"upper":[2.615,3.8675,1.3675000000000002,3.0375,2.5650000000000004,4.257499999999999,14.02],
"lower":[2.215,2.43,1.22,1.8,1.8450000000000002,2.3850000000000002,2.35],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM","Java JVM","Java JVM","Java JVM","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JVM","Kotlin JVM","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin JVM Snapstart"],
"y":[4.6,4.12,5.22,3.39,3.38,7.29,18.1,17.83,12.12,13.88,9.73,18.85,11.93,16.01,2.33,2.03,2.23,1.61,29.86,21.04,32.49,39.66,42.36,49.42,72.5,68.59,4.95,18.79,54.58,8.85,31.87,19.19,7.12,57.39,5.43,5.04,13.17,40.65,14.12,6.66,17.45,27.8,5.49,30.39,5.47,27.97,5.37,8.68,10.19,6.67,8.93,19.38,7.32,7.84,11.48,18.94,26.25,14.4,9.39]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("tqcgHG");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 55 
 

In [68]:
plotWarmStartsBoxplot(1024)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="e0zPjE"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[2.42,2.92,1.01,3.04,2.75,3.09,6.5649999999999995],
"min":[2.04,2.34,0.76,1.62,1.78,2.1,2.07],
"max":[3.17,5.75,1.42,10.39,3.97,8.44,8.42],
"lower":[2.25,2.525,0.91,1.9649999999999999,2.3425000000000002,2.455,2.515],
"upper":[2.665,4.2325,1.12,6.0649999999999995,3.065,4.869999999999999,6.9725],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM Snapstart","Java JVM Snapstart","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM"],
"y":[10.1,15.52,1.75,1.66,1.61,1.49,1.45,1.61,5.68,5.87,1.49,1.46,19.48,18.11,12.16,11.73,12.65,28.28,32.69,23.95,12.51,12.77,6.31,8.37,4.69,6.28,5.83,6.14,4.97,4.93,6.63]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("e0zPjE");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Native

In [69]:
plotWarmStartsBoxplot(2048)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="AiwcTC"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"functionName",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"duration",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"x",
"name":"Funkcja"
},{
"aesthetic":"y",
"name":"Czas wykonania [ms]"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"color":"x"
},
"stat":"identity",
"data":{
"middle":[1.44,2.8,1.07,4.02,2.12,2.885,2.24],
"min":[1.08,2.3,0.83,2.0,1.78,2.06,2.06],
"max":[3.68,4.98,1.34,12.15,3.68,5.93,2.7],
"lower":[1.28,2.535,1.01,2.33,1.96,2.385,2.15],
"upper":[2.28,3.59,1.1624999999999999,6.625,2.68,3.845,2.375],
"x":["Java JVM","Java JVM Snapstart","Java Native","Kotlin JS","Kotlin JVM","Kotlin JVM Snapstart","Kotlin Native"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"boxplot",
"fatten":0.5
},{
"mapping":{
"x":"x",
"y":"y",
"color":"x"
},
"stat":"identity",
"data":{
"x":["Java JVM","Java JVM","Java JVM","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java JVM Snapstart","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Java Native","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JS","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM","Kotlin JVM Snapstart","Kotlin JVM Snapstart","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native","Kotlin Native"],
"y":[3.95,4.55,3.88,6.19,5.65,5.46,5.19,1.88,1.6,1.58,1.6,14.21,1.55,1.68,1.52,1.45,1.95,1.6,1.69,1.6,1.51,11.89,10.33,10.73,14.1,14.14,13.36,15.02,5.1,5.97,4.39,3.99,7.74,4.96,11.1,6.96,3.44,2.97,4.21,2.8,2.8,2.8]
},
"sampling":"none",
"position":"identity",
"geom":"point"
}]
};
 var plotContainer = document.getElementById("AiwcTC");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 Kotlin JVM Snapstart 
 
 
 
 
 
 
 
 
 Kotlin Native 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 11 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 13 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 Czas wykonania [ms] 
 
 
 
 
 Funkcja 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java JVM Snapstart 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Java Native 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JS 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin JVM 
 
 
 
 
 
 
 
 
 
 
 